Import necessart visualisation library and define inputs simulation and additional vector to add

Display map and use the tool to show plot of the simulation time series.
Additionally add external vector file to overlay on the existsing map

In [1]:
from hat.visualisation import NotebookMap
stations= '~/destinE/outlets_v4.0_20230726_withEFAS.csv'
observations= '~/destinE/destine_observation.nc'

simulations = {
    "i05j": "~/destinE/cama_i05j_stations.nc",
    "i05h": "~/destinE/cama_i05h_stations.nc"
}

config = {
    "station_epsg": 4326,
    "station_id_column_name": "station_id",
    "station_filters":"station_id_num <= 5000",
    "station_coordinates": ["StationLon", "StationLat"],
    "obs_var_name": "obsdis",
    "sim_var_name": "dis"

}

statistics = {
    "i05j": "~/destinE/statistics_i05j.nc",
    "i05h": "~/destinE/statistics_i05h.nc"
}


map = NotebookMap(config, stations, observations, simulations, stats=statistics)

Found 49 common stations


In [3]:
map.mapplot(colorby='kge', sim='i05h')

Initialising ipyleaflet
Initialising plotly
Creating stations markers


In [5]:
map.sim_ds

{'i05j': <xarray.Dataset>
 Dimensions:  (time: 10228, station: 49)
 Coordinates:
   * time     (time) datetime64[ns] 2016-01-01T06:00:00 ... 2023-01-01
   * station  (station) object 'G10058' 'G10065' 'G10020' ... 'G10003' 'G10052'
 Data variables:
     dis      (time, station) float32 ...,
 'i05h': <xarray.Dataset>
 Dimensions:  (time: 10228, station: 49)
 Coordinates:
   * time     (time) datetime64[ns] 2016-01-01T06:00:00 ... 2023-01-01
   * station  (station) object 'G10058' 'G10065' 'G10020' ... 'G10003' 'G10052'
 Data variables:
     dis      (time, station) float32 ...}

In [ ]:
import xarray as xr
import pandas as pd

def find_common_station(self):
        ids = []

        # Append sets of station IDs from datasets to the list
        ids.append(set(self.obs_ds['station'].values))

        for ds in self.sim_ds.values():
            ids.append(set(ds['station'].values))

        ids.append(set(self.stations_metadata[self.station_index]))

        print("Station IDs from datasets:", ids)

        # Find common station IDs among the sets
        common_ids = set.intersection(*ids)
        print("Common Station IDs:", common_ids)

        return list(common_ids)

# Load the observations dataset
obs_ds = xr.open_dataset('~/destinE/station_attributes_with_obsdis06h_197001-202312_20230726_withEFAS.nc')

# Load the simulations datasets
sim_i05j = xr.open_dataset('~/destinE/cama_i05j_stations.nc')



In [ ]:
obs_ds.obsdis.values

In [ ]:
import xarray as xr
import os

simulations = {
    "i05j": "~/destinE/cama_i05j_stations.nc",
    "i05h": "~/destinE/cama_i05h_stations.nc"
}
datasets = {}
datasets = {}
for exp, path in simulations.items():
    # Expanding the tilde
    expanded_path = os.path.expanduser(path)
    
    if os.path.isfile(expanded_path):  # Check if it's a file
        ds = xr.open_dataset(expanded_path)
    elif os.path.isdir(expanded_path):  # Check if it's a directory
        # Handle the case when it's a directory; 
        # assume all .nc files in the directory need to be combined
        files = [f for f in os.listdir(expanded_path) if f.endswith('.nc')]
        ds = xr.open_mfdataset([os.path.join(expanded_path, f) for f in files], combine='by_coords')
    else:
        raise ValueError(f"Invalid path: {expanded_path}")
    datasets[exp] = ds
